In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import json

#function to read ../auth_config.json into a object and return it
def get_config():
    with open('../auth_config.json') as f:
        data = json.load(f)
    return data


ssh_config = get_config()

In [2]:
import paramiko
import pandas as pd

def download_file_via_ssh(hostname, username, password, remote_path, local_path):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, username=username, password=password)
    sftp = ssh.open_sftp()
    sftp.get(remote_path, local_path)
    sftp.close()
    ssh.close()



csv_files = {
    '5.9.3.csv': '/root/passivbot_5.9.3/run_config.csv',
    '5.9.0.csv': '/root/passivbot_5.9.0/run_config.csv',
}

# Create an empty DataFrame
csv_data = pd.DataFrame()
for csv_file in csv_files:
    download_file_via_ssh(ssh_config['ip'], ssh_config['user'], ssh_config['password'],
                          csv_files[csv_file], csv_file)
    new_data = pd.read_csv(csv_file)
    new_data['version'] = csv_file.replace('.csv', '')
    csv_data = pd.concat([csv_data, new_data], ignore_index=True)

csv_data

,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,account,version
0,0,ETH,long,n,m,0.08,0.00,binance_01,5.9.3
1,1,BNB,long,n,m,0.08,0.00,binance_01,5.9.3
2,2,ADA,long,n,m,0.08,0.00,binance_01,5.9.3
3,3,KSM,long,n,m,0.08,0.00,binance_01,5.9.3
4,4,LTC,long,n,m,0.08,0.00,binance_01,5.9.3
5,5,ETC,long,n,m,0.08,0.00,binance_01,5.9.3
6,6,XMR,long,n,m,0.08,0.00,binance_01,5.9.3
7,7,TRX,long,n,m,0.08,0.00,binance_01,5.9.3
8,8,ALGO,long,n,m,0.08,0.00,binance_01,5.9.3
9,9,COMP,long,n,m,0.08,0.00,binance_01,5.9.3


In [5]:
total_long_exposure = csv_data[(csv_data['long_mode'] == 'n')]['long_exposure'].sum()
total_short_exposure = csv_data[(csv_data['short_mode'] == 'n')]['short_exposure'].sum()

exposure = total_short_exposure + total_long_exposure

print(f'long exposure: {total_long_exposure}, short exposure: {total_short_exposure}, total exposure: {exposure})')



long exposure: 2.0100000000000002, short exposure: 0.5199999999999999, total exposure: 2.5300000000000002)


In [6]:
total_long_exposure_version_5_9_3 = csv_data[(csv_data['long_mode'] == 'n') & (csv_data['version'] == '5.9.3')]['long_exposure'].sum()
total_long_exposure_version_5_9_0 = csv_data[(csv_data['long_mode'] == 'n') & (csv_data['version'] == '5.9.0')]['long_exposure'].sum()
total_short_exposure_version_5_9_0 = csv_data[(csv_data['short_mode'] == 'n') & (csv_data['version'] == '5.9.0')]['short_exposure'].sum()

print(f'long exposure 5.9.3: {total_long_exposure_version_5_9_3}, long exposure 5.9.0: {total_long_exposure_version_5_9_0}, short exposure 5.9.0: {total_short_exposure_version_5_9_0})')



long exposure 5.9.3: 0.9599999999999999, long exposure 5.9.0: 1.0500000000000005, short exposure 5.9.0: 0.5199999999999999)


In [16]:
total_wallet = 607
margin = 7
coins = csv_data.shape[0]
longs = csv_data[(csv_data['long_mode'] == 'n')].shape[0]
shorts = csv_data[(csv_data['short_mode'] == 'n')].shape[0]

ratio = shorts/longs

total_exposure = longs+shorts
total_wallet_with_margin = total_wallet * margin
per_entry = total_wallet_with_margin / total_exposure
money_cost = total_wallet / total_exposure

exposure = money_cost / margin

xxx = (exposure / total_wallet) * 100

print(f'coins: {coins}, longs:{longs}, shorts:{shorts} , ratio:{ratio}')
xxx

KeyboardInterrupt: 